In [ ]:
import qiskit
from qiskit import QuantumCircuit, BasicAer, Aer, transpile, execute
import qiskit.providers.aer.library
from qiskit.quantum_info import random_statevector
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.opflow import I, X, Y, Z, expectations
from qiskit.algorithms.optimizers import ADAM
from qiskit.circuit import ParameterVector
from qiskit.algorithms import VQE

from matplotlib import pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl

from random import random
import numpy as np
from statistics import mean
from scipy.optimize import minimize
from tqdm import tqdm
import itertools
import copy
from time import time

In [ ]:
def add_layers(circuit, params):
    
    for p in range(0, len(params), 3):
        
        beta = params[p]
        theta = params[p + 1]
        gamma = params[p + 2]
        
        circuit.rxx(beta,14,15)
        circuit.rxx(beta,6,7)
        circuit.rxx(beta,0,8)
        circuit.rxx(beta,1,9)
        circuit.rxx(beta,13,5)
        circuit.rxx(beta,12,4)
        circuit.rxx(beta,3,2)
        circuit.rxx(beta,11,10)
        circuit.barrier()
        circuit.ryy(theta,15,7)
        circuit.ryy(theta,14,6)
        circuit.ryy(theta,8,9)
        circuit.ryy(theta,0,1)
        circuit.ryy(theta,5,4)
        circuit.ryy(theta,13,2)
        circuit.ryy(theta,2,10)
        circuit.ryy(theta,3,11)
        circuit.barrier()
        circuit.rzz(gamma,15,12)
        circuit.rzz(gamma,6,5)
        circuit.rzz(gamma,1,2)
        circuit.rzz(gamma,8,11)
        circuit.barrier()
        circuit.rzz(gamma,7,0)
        circuit.rzz(gamma,9,14)
        circuit.rzz(gamma,3,4)
        circuit.rzz(gamma,10,13)
        circuit.barrier()
        
    return circuit

In [ ]:
def create_circuit(num_qubits, params):
    
    circuit = QuantumCircuit(num_qubits)
    
    #initialize state
    for i in range(0, num_qubits):
        circuit.h(i)
    circuit.barrier()
    
    add_layers(circuit, params)
    
    return circuit

def create_random_param(depth):
    
    params = []
    for _ in range(depth * 3):
        params.append(random() * 2 * np.pi)
        
    return params

In [167]:
"""Here we define the cost function"""

def xx_expectation_value(counts, shots): 
    def bitstring(string):
        energy = 0
        energy -= (1 if string[14] == string[15] else -1)
        energy -= (1 if string[6] == string[7] else -1)
        energy -= (1 if string[0] == string[8] else -1)
        energy -= (1 if string[1] == string[9] else -1)
        energy -= (1 if string[13] == string[5] else -1)
        energy -= (1 if string[12] == string[4] else -1)
        energy -= (1 if string[3] == string[2] else -1)
        energy -= (1 if string[11] == string[10] else -1)
        return energy
    
    energy = 0
    for k,v in counts.items():
        energy += bitstring(k) * (v / shots)
    return energy

def yy_expectation_value(counts, shots):
    def bitstring(string):
        energy = 0
        energy -= (1 if string[15] == string[7] else -1)
        energy -= (1 if string[14] == string[6] else -1)
        energy -= (1 if string[8] == string[9] else -1)
        energy -= (1 if string[0] == string[1] else -1)
        energy -= (1 if string[5] == string[4] else -1)
        energy -= (1 if string[13] == string[2] else -1)
        energy -= (1 if string[2] == string[10] else -1)
        energy -= (1 if string[3] == string[11] else -1)
        return energy
    
    energy = 0
    for k,v in counts.items():
        energy += bitstring(k) * (v / shots)
    return energy

def zz_expectation_value(counts, shots):
    def bitstring(string):
        energy = 0
        energy -= (1 if string[15] == string[12] else -1)
        energy -= (1 if string[6] == string[5] else -1)
        energy -= (1 if string[1] == string[2] else -1)
        energy -= (1 if string[8] == string[11] else -1)
        energy -= (1 if string[7] == string[0] else -1)
        energy -= (1 if string[9] == string[14] else -1)
        energy -= (1 if string[3] == string[4] else -1)
        energy -= (1 if string[10] == string[13] else -1)
        return energy
    
    energy = 0
    for k,v in counts.items():
        energy += bitstring(k) * (v / shots)
    return energy

def cost_function(params):

    num_qubits = 16
    shots = 16384
    backend = Aer.get_backend('qasm_simulator')

    # Z basis measurement
    circuit = create_circuit(num_qubits, params)  
    circuit.measure_all()
    job = execute(circuit, backend, shots = shots)
    result = job.result()
    counts = result.get_counts(circuit)
    zz_exp = zz_expectation_value(counts, shots)

    # X basis measurement
    circuit = create_circuit(num_qubits, params)  
    for i in range(num_qubits):
        circuit.h(i)
    circuit.measure_all()
    job = execute(circuit, backend, shots = shots)
    result = job.result()
    counts = result.get_counts(circuit)
    xx_exp = xx_expectation_value(counts, shots)
    
    # Y Basis measurement
    circuit = create_circuit(num_qubits, params)
    for i in range(num_qubits):
        circuit.sdg(i)
        circuit.h(i)
    circuit.measure_all()
    job = execute(circuit, backend, shots = shots)
    result = job.result()
    counts = result.get_counts(circuit)
    yy_exp = yy_expectation_value(counts, shots)

    return (zz_exp + xx_exp + yy_exp) * -1

"""Find the energy of the shifted parameter using our cost function"""
def find_shifted_energy(index, shift, params):
    params_clone = copy.deepcopy(params)
    params_clone[index] = params[index] + shift
    shifted_energy = cost_function(params_clone)
    return shifted_energy

In [168]:
"""Find the coefficients for the newly defined cost function"""
def find_coefficient_vector(params,index):
    S = 8 #how many times this parameter appears in the circuit
    angle_shifts = (2*np.pi / (2*S+1)) * np.arange(2*S+1) #set of angles from 0 to 2*pi to shift by (2*S+1 cuts)
    shifted_energies = np.zeros(2*S+1)
    for i in tqdm(range(len(angle_shifts))):
        shifted_energies[i] = find_shifted_energy(index, angle_shifts[i], params)  
    coefficient_vector = np.fft.rfft(shifted_energies)
    return coefficient_vector

"""New cost function is defined as L"""
def array_cost_function(index, angle_array, params):
    coeffs = find_coefficient_vector(params, index)
    energy_array = np.zeros(len(angle_array)) + coeffs[0]
    for i in range(len(coeffs)-1):
        energy_array += 2 * np.real(coeffs[i + 1] * np.exp(1j * (i + 1) * (angle_array)))
    energy_array /= 2 * len(coeffs) - 1
    return energy_array

def single_cost_function(parameter, coeffs):
    energy = float(coeffs[0])
    for i in range(len(coeffs)-1):
        energy += 2 * np.real(coeffs[i+1] * np.exp(1j * (i+1) * parameter))
    energy /= 2 * len(coeffs) - 1
    return float(energy)

def update(params, index, coeffs):
    parameter = params[index]
    result = minimize(single_cost_function, parameter, args=(coeffs), method = 'L-BFGS-B')
    updated_params=copy.deepcopy(params)
    updated_params[index] = result['x']
    updated_energy = result['fun']
    return updated_params, updated_energy

In [169]:
DEPTH = 1
MAX_RUNS = 2

energy_array = []
error = 1000 #some arbitrary large number to meet loop condition
num_runs = 0
params=np.zeros(DEPTH*3)

start_time = time()
while error > 5e-4 and num_runs < MAX_RUNS:
    num_runs += 1
    print("On loop #" + str(num_runs))
    
    for i in range(len(params)):
        if i == 0:
            energy = cost_function(params)
        index = i
        coeffs = find_coefficient_vector(params, index)
        params, energy = update(params, index, coeffs)
        energy_array.append(energy)
        print("params: ", params)
        print("energy: ", cost_function(params))
#         error = energy_array[-1] - exact
#         print("error: ", error)
        print("Time elapsed: ", time() - start_time)

On loop #1


100%|██████████| 17/17 [00:11<00:00,  1.50it/s]
<ipython-input-168-d935c4ebef84>:21: ComplexWarning: Casting complex values to real discards the imaginary part
  energy = float(coeffs[0])


params:  [1.00548935 0.         0.        ]


  0%|          | 0/17 [00:00<?, ?it/s]

energy:  8.019775390625
time elapsed:  12.681723356246948


...

params:  [ 0.99921632  4.16972564 -4.72382967]


  0%|          | 0/17 [00:00<?, ?it/s]

energy:  0.0389404296875
time elapsed:  61.800235986709595


100%|██████████| 17/17 [00:11<00:00,  1.46it/s]


params:  [ 0.99921632  4.16972564 -0.02955108]
energy:  4.986328125
time elapsed:  74.10702204704285
